In [ ]:
#teste de normalidade dos tratamentos
import numpy as np
from scipy.stats import shapiro, gaussian_kde
data_box_abs = [df_diff[diff_trats].dropna().values for trat in trats]
shapiro_results = {}
for i, data in enumerate(data_box_abs):
    stat, p_value = shapiro(data)
    shapiro_results[trats[i]] = (stat, p_value) # Armazena o resultado do teste
    print(f'Tratamento: {trats[i]}, Estatística de Shapiro-Wilk: {stat:.4f}, p-valor: {p_value:.4f}')

# Análise de densidade para dados não normais   
import matplotlib.pyplot as plt
fig, axs = plt.subplots(len(data_box_abs), 1, figsize=(8, 4 * len(data_box_abs)))
for i, data in enumerate(data_box_abs):
    data = np.array(data)
    data = data[~np.isnan(data)]
    if len(data) > 1:
        kde = gaussian_kde(data)
        min_val, max_val = np.percentile(data, [25, 75])
        x_vals = np.linspace(min_val - 3 * kde.factor, max_val + 3 * kde.factor, 1000)
        axs[i].plot(x_vals, kde(x_vals), label='KDE')
        axs[i].hist(data, bins=30, density=True, alpha=0.5, label='Histograma')
        axs[i].set_title(f'Análise de Densidade - {trats[i]}')
        axs[i].set_xlabel('Discrepância Absoluta (m)')
        axs[i].set_ylabel('Densidade')
        axs[i].legend()
plt.tight_layout()
plt.show()
plt.savefig('densidade_discrepancias_absolutas.png', dpi=300, bbox_inches='tight')

#Teste de homocedasticidade de Levene
from scipy.stats import levene
levene_results = levene(*data_box_abs)
print(f'Teste de Levene - Estatística: {levene_results.statistic:.4f}, p-valor: {levene_results.pvalue:.4f}')
'''
#Como os dados não são normais e não possuem homocedasticidade, será utilizado o teste não paramétrico de Kruskal-Wallis
from scipy.stats import kruskal
kruskal_results = kruskal(*data_box_abs)
print(f'Teste de Kruskal-Wallis - Estatística: {kruskal_results.statistic:.4f}, p-valor: {kruskal_results.pvalue:.4f}')
#Teste post-hoc de Dunn com correção de Bonferroni
import scikit_posthocs as sp        
dunn_results = sp.posthoc_dunn(data_box_abs, p_adjust='bonferroni')
dunn_results.index = trats
dunn_results.columns = trats
print('Teste post-hoc de Dunn com correção de Bonferroni:')
print(dunn_results)
dunn_results.to_excel('dunn_posthoc_results.xlsx')

'''

In [ ]:
#GRAFICOS COM PYQTGRAPH
import pyqtgraph as pg
plot_widget = pg.PlotWidget()
plot_widget.plot([1,2,3], [4,5,6])
layout.addWidget(plot_widget)
import pyqtgraph as pg
import numpy as np
from pyqtgraph.Qt import QtWidgets

# Create the application
app = QtWidgets.QApplication([])

# Create window and layout
win = pg.GraphicsLayoutWidget(show=True)
win.setWindowTitle('Boxplots de Discrepâncias')

# Create plots
p1 = win.addPlot(title="Discrepâncias")
p2 = win.addPlot(title="Discrepâncias Absolutas")
win.nextRow()

# Convert dataframes to numpy arrays for plotting
data_diff = [df_diff[col].values for col in df_diff.columns]
data_abs = [df_diff_abs[col].values for col in df_diff_abs.columns]

# Create box plots
box1 = pg.BoxPlotItem(data=data_diff, labels=df_diff.columns.tolist())
box2 = pg.BoxPlotItem(data=data_abs, labels=df_diff_abs.columns.tolist())

p1.addItem(box1)
p2.addItem(box2)

# Set labels
p1.setLabel('left', 'Discrepância (m)')
p2.setLabel('left', 'Discrepância Absoluta (m)')

# Show the plots
if __name__ == '__main__':
    pg.exec()